In [2]:
import pandas as pd 
import numpy as np
import jieba
import jieba.posseg as pseg

## 数据读取

In [12]:
df1 = pd.concat([pd.read_excel('data/2018-2019茂名（含自媒体）.xlsx', sheet_name='游记攻略'), pd.read_excel('data/2020-2021茂名（含自媒体）.xlsx', sheet_name='游记攻略')], ignore_index=True)
df2 = pd.concat([pd.read_excel('data/2018-2019茂名（含自媒体）.xlsx', sheet_name='酒店评论'), pd.read_excel('data/2020-2021茂名（含自媒体）.xlsx', sheet_name='酒店评论')], ignore_index=True)
df3 = pd.concat([pd.read_excel('data/2018-2019茂名（含自媒体）.xlsx', sheet_name='景区评论'), pd.read_excel('data/2020-2021茂名（含自媒体）.xlsx', sheet_name='景区评论')], ignore_index=True)
df4 = pd.concat([pd.read_excel('data/2018-2019茂名（含自媒体）.xlsx', sheet_name='餐饮评论'), pd.read_excel('data/2020-2021茂名（含自媒体）.xlsx', sheet_name='餐饮评论')], ignore_index=True)

In [4]:
df1.head()

,游记ID,城市,游记标题,发布时间,出发时间,出行天数,人物,人均费用,正文
0,1001,茂名,漫游中国之三、广东(下)……茂名、惠州、河源、和平、韶关,2018-05-02 07:46,NaN,NaN,NaN,NaN,1、茂名放鸡岛\n放鸡岛 （原名湾舟岛，又名汾洲岛）是位于 广东 省 茂名 市 电白 区博贺...
1,1002,茂名,粤西美食| 茂名，心中的美食城（不定时更新）,2018-10-15 22:21,NaN,NaN,NaN,NaN,【茂名在哪】\n我的大 茂名 位于 广东 西部，简称粤西，很多人都不知道 茂名 是哪，所以我...
2,1003,茂名,【茂名】三日游记,2018-11-15 11:38,2018-08-25,3 天,一个人,400RMB,行程总览\n【地点】\n茂名\n\n【时间】\n3天（实际游玩时间约2天半）\n\n【交通】...
3,1004,茂名,几分钟和你吃遍茂名美食-粤西茂名家乡发展有你我,2018-11-30 18:12,2018-11-30,1 天,和朋友,50RMB,前言\n几分钟真的吃不遍 茂名 ，题目只是一个噱头 🔥🔥这是我第一次写家乡的游记 很多店我觉...
4,1005,茂名,寻找茂名的历史——周末游高州、化州,2018-12-06 08:09,2018-11-24,2 天,一个人,800RMB,茂名 原本相对于 广州 而言，交通稍差，不适合周末二日游。随着江湛铁路的开通，交通已经不成问...


## 数据预处理

### 缺失值填充

In [5]:
df1[['人物','人均费用']]

,人物,人均费用
0,NaN,NaN
1,NaN,NaN
2,一个人,400RMB
3,和朋友,50RMB
4,一个人,800RMB
...,...,...
289,NaN,NaN
290,NaN,NaN
291,家庭出游,250RMB
292,家庭出游,500RMB


In [6]:
df1['人均费用_'] = df1['人均费用'].apply(lambda s:str(s).split('RMB')[0])

In [7]:
df1['人均费用_'] = df1['人均费用_'].astype(float)

In [8]:
df1['人均费用_'].fillna(df1['人均费用_'].mean(), inplace=True)

### 去除换行符

In [10]:
df5.columns

Index(['文章ID', '公众号标题', '发布时间', '正文'], dtype='object')

In [13]:
df1['正文'] = df1['正文'].apply(lambda text: str(text).replace('\n', '').replace(' ', ''))
df2['评论内容'] = df2['评论内容'].apply(lambda text: str(text).replace('\n', ''))
df3['评论内容'] = df3['评论内容'].apply(lambda text: str(text).replace('\n', ''))
df4['评论内容'] = df4['评论内容'].apply(lambda text: str(text).replace('\n', ''))

### 分词处理

In [14]:
df1.head(1)

,游记ID,城市,游记标题,发布时间,出发时间,出行天数,人物,人均费用,正文
0,1001,茂名,漫游中国之三、广东(下)……茂名、惠州、河源、和平、韶关,2018-05-02 07:46,NaN,NaN,NaN,NaN,1、茂名放鸡岛放鸡岛（原名湾舟岛，又名汾洲岛）是位于广东省茂名市电白区博贺镇东南14.5公里...


In [15]:
df2.head(1)

,酒店评论ID,城市,酒店名称,评论日期,评论内容,入住日期,入住房型
0,1001,茂名,茂名君悦商务酒店,2018-12-02,干净卫生服务好,2018-12-02,标双


In [16]:
df3.loc[5, '评论内容']

'山很高，空气清新，环境还可以，最好玩的还是滑道'

In [17]:
df4.head(1)

,餐饮评论ID,城市,餐饮名称,评论日期,评论内容,标题
0,1001,茂名,盛香烧鹅（东方市场店）,2018-01-01,很好吃推荐！,主食3选1，有赠品


## 划分训练和预测数据数据

## 实体标注

In [17]:
# 实体标注
def text_code(data=None):
    text_code_list = []
    for text in data:
        print('本次标注的内容为：\n', text)
        while True:
            entity = str(input())
            if entity == '0':  # 找完实体后输入0退出输入
                break
            else:
                text = str(text).replace(entity, 'B' + 'I'*(len(entity)-1))  # 实体开始为B，中间部分为I

        for w in list(text):  # 标注非实体的词为O
            if w not in 'BIO':
                text = text.replace(w, 'O')
        text_code_list.append(text)
        print('实体标注结果为：\n', text)

    return text_code_list

In [20]:
# text_code_list1 = text_code(df3['评论内容'])
# print(text_code_list1)

In [21]:
# df3['实体编码'] = text_code_list1

In [22]:
# df3.head()

In [23]:
# df3.to_csv('data/景区评论.csv', index=False)

In [24]:
# text_code_list2 = text_code(df1['正文'])
# print(text_code_list2)

## 茂名市景区

In [25]:
bd_df = pd.read_csv('data/百度地图茂名市景区.txt', header=None)
bd_df.head()

,0
0,高州仙人洞景区
1,粤龙山风景区
2,笔架山森林公园
3,露天矿生态公园
4,莲花湖


In [26]:
gd_df = pd.read_csv('data/高德地图茂名市景区.txt', header=None)
gd_df.head()

,0
0,观珠文化广场
1,浮山岭风景区
2,市民公园
3,根子浮山冼太庙
4,鸡藤坡娱乐文化广场


In [27]:
# 合并去重景区名称
scenic = set(bd_df.iloc[:, 0].tolist() + gd_df.iloc[:, 0].tolist())
scenic

{'1号湖公园',
 '丁堡森林公园',
 '丁颖荔枝森林公园',
 '万亩红树林景区',
 '万寿庵',
 '万年昌村文化广场‘',
 '三仙庙',
 '三佳公园',
 '三侯大庙',
 '三圣帝宫',
 '三圣庙',
 '三太庙',
 '三官仔岭顶观瀑台',
 '三尊庙',
 '三帝宫',
 '三更劝学公园',
 '三木埇文化广场',
 '三洲广场',
 '三清古庙',
 '三甲观海亭',
 '上关村广场',
 '上坡村安宁堂',
 '上坡村文化广场',
 '上垌文化广场',
 '上庵黑泥塘党建公园',
 '上庵黑泥塘村文化广场',
 '上村文化广场',
 '上泗水村文化广场',
 '下坡古庙',
 '下塘村文化广场',
 '下大德村文化广场',
 '下大茶旧戏台广场',
 '下山口村文化广场',
 '下村海埠',
 '下盐仓村爱心公园',
 '下郭游乐园',
 '下郭美食文化广场',
 '世湛刘公宅',
 '东坡村党建广场',
 '东坡村广场',
 '东头村文化广场',
 '东安庙',
 '东岸横圩冼太庙',
 '东岸镇沙坡森林公园',
 '东方红花海',
 '东湖公园',
 '东镇慈孝文化园',
 '丫江岛',
 '中共化县支部成立遗址',
 '中共电白县支部旧址纪念馆',
 '中共茂电信工委高圳车活动旧址',
 '中华化橘红第一村文化广场',
 '中华第一冼太庙',
 '中国第一滩',
 '中国第一滩-凉亭',
 '中国第一滩旅游度假区',
 '中国道教茂南区罗冼庙办公室',
 '中垌石子塘广场',
 '中垌簕筒文化广场',
 '中山村广场',
 '中心台大森林',
 '中心广场',
 '中祠堂',
 '丰堂大王庙',
 '丰堂文化广场',
 '丹山村文化广场',
 '主境顺调',
 '丽珠社文化广场',
 '乐垌大道牌坊',
 '乐垌文化中心广场',
 '乐岭文化广场',
 '九仕坡村文化广场',
 '九仕坡村祠堂边',
 '九龙山旅游区',
 '九龙庙',
 '书房仔村文化广场',
 '书房岭土地庙',
 '云径庙',
 '云海飞车',
 '云炉中心文化广场',
 '云炉公园',
 '云炉庙',
 '五道馆',
 '井头坡村文化广场',
 '人民公园',
 '人民广场',
 '仙人洞森林公园',
 '仙人洞自然风景旅游区售票处',
 '仙姑庙',
 '

In [28]:
len(scenic)

937

## 酒店、景区和餐饮实体合集

In [44]:
jq = pd.read_csv('data/景区评论.csv')
cy = pd.read_csv('data/餐饮评论.csv')

In [13]:
# 实体识别
def find_entity(conts=None, codes=None):
    entitys = set()
    for cont, code in zip(conts, codes):
        entity = ''
        for w, c in zip(list(cont), list(code)):
            if c == 'B' or c == 'I':
                entity += w
            else:
                if entity != '':
                    entitys.add(entity)
                entity = ''
    return list(entitys)

In [53]:
entitys_jq = find_entity(jq['评论内容'], jq['实体编码'])
entitys_cy = find_entity(cy['评论内容'].astype(str), cy['实体编码'].astype(str))

In [54]:
entitys_jq

['望海楼',
 '湛江大桥',
 '博物馆',
 '红树林',
 '瞭望厅',
 '湖光岩',
 '雷州西湖',
 '海滨公园',
 '雷州古城',
 '西湖公园',
 '露天矿生态公园',
 '水东湾',
 '国家山地公园',
 '三元塔',
 '十里军港',
 '麻斜岛',
 '雷祖祠',
 '海滩',
 '龙湾生态旅游区',
 '游乐场',
 '杏岛',
 '玻璃桥',
 '摩天轮',
 '阳江海上丝绸之路博物馆',
 '十贤祠',
 '渔岛度假村',
 '东南码头',
 '广州海上丝绸之路博物馆',
 '海陵岛',
 '赤坎老街']

In [55]:
entitys_cy

['叉烧饭',
 '牛肉',
 '炒牛河',
 '生蚝',
 '炒乌冬',
 '牛肚',
 'UNPOCO柏蔻',
 '凉拌菜',
 '榴莲蛋糕',
 '甜品',
 '刺身',
 '日式蟹肉卷',
 '海鲜',
 '无花果',
 '甜点',
 '寿司',
 '油条碎',
 '脆皮热狗卷',
 '自助菜',
 '早茶',
 '菠萝包',
 '关东煮',
 '蜂蜜芥末',
 '煲仔饭',
 '佛罗伦萨式5J烩牛肚',
 '牛排',
 '小炒',
 '蔬菜',
 '肉松',
 '面包体',
 '面包',
 '饮料',
 '炸鸡',
 '紫菜肉松卷',
 '热狗',
 '蛋挞',
 '薯条',
 '黄油',
 '和牛盖饭',
 '高钙大骨鸳鸯锅',
 '榴莲千层',
 '奶油',
 '酱汁',
 '米饭',
 '火腿',
 '烧鸭饭',
 '菠萝皮',
 '拉面',
 '泡芙',
 '意粉拌酱汁',
 '粗薯',
 '无花果萨拉米披萨']

In [57]:
jd = df2['酒店名称'].tolist()

In [10]:
df2.to_csv('data/酒店评论.csv', index=False)

In [58]:
scenics = []
scenics.extend(list(scenic))
scenics.extend(jq)
scenics.extend(cy)
scenics.extend(jd)

In [59]:
df1['正文']

0     一元复始，万象更新，庆祝元旦猪杂先生：庆祝元旦，从早餐美味开始油甘窝，黑马国际俱乐部金塘，谭...
1     水东城，位于南海水东湾畔，面积49.2平方公里，是广东省茂名市电白区区府所在地。水东原为滨海...
2     前言水东，是广东省茂名市电白区区府所在地。水东原为滨海渔村，明代开始逐渐发展成为电白的最大商...
3     元旦日后，阳光静好，迎君来储良杏花村佳话，琴瑟永和谐分界储良，杏花村，莫五姑的家乡。莫秀英（...
4     今日行程：茂名【幸福大院出发】——油甘窝——化州平垌【湖艳村，藏龙湾】——南盛山合【太阳部落...
5     到信宜汽车总站后交通车7元直达镇隆镇，2021.01.21日下午到达窦州温泉。窦州温泉又称“...
6     到罗平镇整晚小雨躲进了一家装修超市二楼，2021.01.21日早上从罗定罗平镇坐车到达茂名新...
7     鳌头镇位于茂名市茂南区，距市区中心广场15公里，东、南毗邻电白区，西接吴川市。全镇总面积56...
8     博贺湾（又称博贺港），位于广东省茂名市电白区博贺镇和树仔镇之间，是南海的一个港湾，呈东西走向...
9     2021年2月6日，距牛年春节还有六天，天气非常晴朗，最高气温达到27度。这是旅行的好时机，...
10    自从回到水东以后，一直想目睹水东湾跨海大桥的雄姿，今天终于如愿以偿。从2012年立项和建设到...
11    来啦！推荐一个茂名巨小众景点：茂名电白架海／白色拖尾沙滩～～～这是场即兴的心动旅程！💓这里是...
12    简介：天马山位于信宜市北界镇，北与广西容县接壤，西与广西北流毗邻，景区距离信宜市区仅38公里...
13    嫁到茂名的女娃，这个春节，毫无意外的跟着老公孩子回来茂名，不过今年春节是有海有风有太阳有贝壳...
14    今年隔壁开了新高速，大大节省了路上的时间，于是趁着新年，全家到茂名去浪一浪。从前坐高铁过去都...
15    2021年3月12下午3点钟从广州出发到湛江吴川大概是花了6个小时左右，到达吴川已经是晚上9...
16    新湖公园+茂名博物馆小黄车绿道游骑着我心爱的小黄车～它永远不会停电～时间：3：00PM，天气...
17    今天打卡雅米乐园！平日里在家的娃真的是被手机、ipad吸住了，所以一到放假就想带他们

In [60]:
text_code_bi = []
for cont in df1['正文']:
    for s in scenics:
        if s in cont:
            cont = str(cont).replace(s, 'B' + 'I'*(len(s)-1))
    text_code_bi.append(cont)

In [61]:
len(text_code_bi)

50

In [62]:
text_code_list3 = []
for cont in text_code_bi:     
    for w in list(cont):
        if w not in 'BI0':
            cont = cont.replace(w, 'O')
    text_code_list3.append(cont)

In [63]:
text_code_list3

['...']

In [64]:
len(text_code_list3)

50

In [65]:
df1.shape

(50, 9)

In [66]:
df1['实体编码'] = text_code_list3
df1.to_csv('data/游记攻略.csv', index=False)

## 游记攻略完善标注

In [2]:
df_yj = pd.read_csv('data/游记攻略.csv')
df_yj.head()

,游记ID,城市,游记标题,发布时间,出行天数,人物,人均费用,正文,人均费用_,实体编码
0,游记攻略-001,茂名,2021-1-1 一路走来﻿皆有味道 ﻿送鼠迎牛庆贺元旦,2021-01-02 10:49,1 天,和朋友,180RMB,一元复始，万象更新，庆祝元旦猪杂先生：庆祝元旦，从早餐美味开始油甘窝，黑马国际俱乐部金塘，谭...,180.0,OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...
1,游记攻略-002,茂名,水东忠良街游记,2021-01-04 07:44,1 天,一个人,20RMB,水东城，位于南海水东湾畔，面积49.2平方公里，是广东省茂名市电白区区府所在地。水东原为滨海...,20.0,OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...
2,游记攻略-003,茂名,水东滨海游记,2021-01-04 21:00,1 天,和朋友,20RMB,前言水东，是广东省茂名市电白区区府所在地。水东原为滨海渔村，明代开始逐渐发展成为电白的最大商...,20.0,OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...
3,游记攻略-004,茂名,2021-1-2 借问乡旅何处好 高佬推荐杏花村,2021-01-02 21:24,1 天,和朋友,100RMB,元旦日后，阳光静好，迎君来储良杏花村佳话，琴瑟永和谐分界储良，杏花村，莫五姑的家乡。莫秀英（...,100.0,OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...
4,游记攻略-005,茂名,2021-1-10 湖艳藏龙湾深埇大地大王岭绿野仙踪 南盛过山合太阳部落上宾塘知了书院,2021-01-11 08:58,1 天,和朋友,280RMB,今日行程：茂名【幸福大院出发】——油甘窝——化州平垌【湖艳村，藏龙湾】——南盛山合【太阳部落...,280.0,OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...


In [5]:
df_yj['正文'].head()

0    一元复始，万象更新，庆祝元旦猪杂先生：庆祝元旦，从早餐美味开始油甘窝，黑马国际俱乐部金塘，谭...
1    水东城，位于南海水东湾畔，面积49.2平方公里，是广东省茂名市电白区区府所在地。水东原为滨海...
2    前言水东，是广东省茂名市电白区区府所在地。水东原为滨海渔村，明代开始逐渐发展成为电白的最大商...
3    元旦日后，阳光静好，迎君来储良杏花村佳话，琴瑟永和谐分界储良，杏花村，莫五姑的家乡。莫秀英（...
4    今日行程：茂名【幸福大院出发】——油甘窝——化州平垌【湖艳村，藏龙湾】——南盛山合【太阳部落...
Name: 正文, dtype: object

In [6]:
df_yj['实体编码'].head()

0    OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...
1    OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...
2    OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...
3    OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...
4    OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...
Name: 实体编码, dtype: object

In [26]:
cont_new = []
for text, code in zip(df_yj['正文'].tolist(), df_yj['实体编码'].tolist()):
    new_text = ''
    for w, c in zip(list(text), list(code)):
        if c == 'B':
            new_text += 'B'
        elif c == 'I':
            new_text += 'I'
        else:
            new_text += w
    cont_new.append(new_text)

In [27]:
df_yj['标注好的正文'] = cont_new

In [28]:
df_yj.head()

,游记ID,城市,游记标题,发布时间,出行天数,人物,人均费用,正文,人均费用_,实体编码,标注好的正文
0,游记攻略-001,茂名,2021-1-1 一路走来﻿皆有味道 ﻿送鼠迎牛庆贺元旦,2021-01-02 10:49,1 天,和朋友,180RMB,一元复始，万象更新，庆祝元旦猪杂先生：庆祝元旦，从早餐美味开始油甘窝，黑马国际俱乐部金塘，谭...,180.0,OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...,一元复始，万象更新，庆祝元旦猪杂先生：庆祝元旦，从早餐美味开始油甘窝，黑马国际俱乐部金塘，谭...
1,游记攻略-002,茂名,水东忠良街游记,2021-01-04 07:44,1 天,一个人,20RMB,水东城，位于南海水东湾畔，面积49.2平方公里，是广东省茂名市电白区区府所在地。水东原为滨海...,20.0,OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...,水东城，位于南海水东湾畔，面积49.2平方公里，是广东省茂名市电白区区府所在地。水东原为滨海...
2,游记攻略-003,茂名,水东滨海游记,2021-01-04 21:00,1 天,和朋友,20RMB,前言水东，是广东省茂名市电白区区府所在地。水东原为滨海渔村，明代开始逐渐发展成为电白的最大商...,20.0,OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...,前言水东，是广东省茂名市电白区区府所在地。水东原为滨海渔村，明代开始逐渐发展成为电白的最大商...
3,游记攻略-004,茂名,2021-1-2 借问乡旅何处好 高佬推荐杏花村,2021-01-02 21:24,1 天,和朋友,100RMB,元旦日后，阳光静好，迎君来储良杏花村佳话，琴瑟永和谐分界储良，杏花村，莫五姑的家乡。莫秀英（...,100.0,OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...,元旦日后，阳光静好，迎君来储良杏花村佳话，琴瑟永和谐分界储良，杏花村，莫五姑的家乡。莫秀英（...
4,游记攻略-005,茂名,2021-1-10 湖艳藏龙湾深埇大地大王岭绿野仙踪 南盛过山合太阳部落上宾塘知了书院,2021-01-11 08:58,1 天,和朋友,280RMB,今日行程：茂名【幸福大院出发】——油甘窝——化州平垌【湖艳村，藏龙湾】——南盛山合【太阳部落...,280.0,OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...,今日行程：茂名【幸福大院出发】——油甘窝——化州平垌【湖艳村，藏龙湾】——南盛山合【太阳部落...


In [29]:
text_code_yj = text_code(df_yj['标注好的正文'])
print(text_code_yj)

本次标注的内容为：
 一元复始，万象更新，庆祝元旦猪杂先生：庆祝元旦，从早餐美味开始油甘窝，黑马国际俱乐部金塘，谭屋，好心湖畔田园综合体牙象陂塘，食为先农业生态园，元旦热闹牙象，木头塘村，焕然一新木头塘，戏剧墙绘，环村小火车大地艺术公园油甘窝村，示范五兄弟别墅，做籺贺元旦美酒白粥，开元启旦童子湾，迎风破浪，凭海听涛宴镜岭，塘下村，冼夫人拍摄基地水东港，跨海大桥跨海大桥，元旦徒步追赶落日，海洋公园徒步线路，美丽墙绘冼夫人剧组，歌美海见面会央视味道，茂名年味，拍摄杀青刷羊煲，红红火火，贺元旦
水东港
0
实体标注结果为：
 OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOBIIOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO
本次标注的内容为：
 水东城，位于南海水东湾畔，面积49.2平方公里，是广东省茂名市电白区区府所在地。水东原为滨海渔村，明代开始有商人落户于此并拓建码头，逐渐发展成为电白的最大商埠。清末至民国期间，水东一直为南路地区的中心圩镇。“水东一圩，,阅阗鳞次，海运巨艚杂舶，商贾往来”（据《电白县志》）。1950年12月，电白县由电城镇迁治水东镇。随着时代发展，水东城镇建设突飞猛进。2014年4月，电白区成立，水东也就成了茂名市区的一部分。2019年水东镇分设为水东街道和电海街道，水东不能再称镇而应叫城了。水东街道辖下的忠良街（已改名忠良社区，为叙述方便，我还是用旧称，大家明白就好了）位于水东城区的东南部，东、南至水东海堤，西至东阳南街，北至城东解放街、澄波街，辖区面积0.75平方公里，有居民小组12个，人口6700余人。该社区历史文化悠久，自古便是繁华商地，辖区内有忠良古街、油地码头、油地公园等景点。2021年1月2日，我到水东忠良街进行了一次考察旅行。一、骑楼古街骑楼是一种近代商住建筑，一楼临街部分建成行人走廊，可以遮阳挡雨。走廊上方为二楼的楼层，犹如二楼“骑在”行人走廊头上，故称为“骑楼”。

水东湾
东湖
红树林湿地公园
0
实体标注结果为：
 OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOBIIIOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOBIIIOOBIIIOOBIIIOOOOOOOOOOOOOOOOOOOOBIIOOOOOOOOOOOOOOOOOOOOOBIIIOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOBIOOBIOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOBIIIOOOOOOOOOOOOOOOOOOOOBIIIOOOOOOOOOOOOOOOBIOOOOOOOOOOOOOOOOOBIIIOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOBIIOOOOOOOOOOOOOOOOOOOOOOOOOOOBIIIOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOBIIIOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO

0
实体标注结果为：
 OOOOOOOOOOOOOOOBIIIOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOBIIIOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOBIIIOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOBIIIOOBIIIOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOBIIIOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOBIIIOOOOOOOOOOBIOOOOOOOOO

0
实体标注结果为：
 OOOOOOOOOOOOOOOOOOOOOOOOOOBIOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOBIOOOOOOOOOOOBIOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO
本次标注的内容为：
 简介：天马山位于信宜市北界镇，北与广西容县接壤，西与广西北流毗邻，景区距离信宜市区仅38公里，离西江温泉度假村23公里，二级硬底化公路直达景区，交通方便。天马山占地26平方公里，旅游区内满目苍翠，空气清新。（摘自百度百科）BIIIIIIII交通春节期间和家人自驾前往天马山，上山的公路比较陡也很弯曲，要注意驾驶，不适宜步行或者骑摩托车进入景区。上山和下山可以乘坐当地的观光车，30多一个人，但是去到山顶有大风车的地方还是要靠步行，建议穿着舒适的运动鞋。BIIIIIIIIBIIIIIIII南国大草原在冬季都是枯黄的，有些可惜，但是另有一番风景。BIIIIIIII绿色的树林和大风车真的非常小清新，手机拍照也能出片。BIIIIIIII在山顶的时候就能到达广西省边界。BIIIIIIII门票成人50，学生票半价，记得带上学生证。饮食上山途中有一个小卖部，可以提供补给，最好自己带足够的水。徒步线路沿着开辟的小路一直走就能到达山顶，但是接近峰顶的时候都是草地，比较滑，要小心。最佳游玩时间我们去的时候已经是草木枯黄的时节，最好选择夏天出游，可以看到绿油油的南国大草原。
天马山
0
实体标注结果为：
 

雅米乐园
0
实体标注结果为：
 OOOOBIIIOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOBIIIOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOBIIIOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOBIIIOOOOOOOOOOOOBIIOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO

...

碉楼
自力村
立园
马降龙
大角湾
北洛湾
0
实体标注结果为：
 OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOBIOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOBIOOBIOOOOOOOOOOOOOOOOBIIOBIOBIIOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOBIIOBIOOOOOOOOOOBIIOBIOBIIOOOOOOOOOOOOOOOOOOBIIOBIOOOOOBIIOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOBIIOOOOOOOOOOOOOOOOOOOOOBIIOBIOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOBIOOOOOOOOBIIOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOBIOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOBIOOOOOBIOOOOOBIOOOOOBIOOOOOBIOOOOOBIOOOBIOOOOOOOOOOOBIOOOOOOOOOOBIOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOBIOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOBIOOOOOBIOOOOOBIOOOOOBIOOOOOOOOOOBIOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOBIOOOOOOOOOOOOOOB

博贺港码头
福鑫酒店
博贺港
海洋公园酒店
野菠萝公园
0
实体标注结果为：
 OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOBIIIIOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOBIIOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOBIIOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOBIIOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOBIIIIIOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOBIIIOOOOOOOOOOOOOBIIIIIOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO

荔枝
0
实体标注结果为：
 OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOBIOOOOOOOOOOOOOOOOOOOOOOOOBIOOBIOOOBIOOOOOOBIOOOOOOOBIBIOOOOOOOOOOOOOOOOOOOOOOOBIOOOOOOOOOOOOOOOOOOOOOOBIOOOOOOOOOOOOOOOOOOOOOOOOOOBIOOOOOOOOOOOOOOOOOOOOOOOOOOBIOOBIBIOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOBIOOOOOOOOOOOOOOOOOOOOOOOOOBIOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO
本次标注的内容为：
 亲子乐园千千万，哪家才是心头好？茂名市区出发20分钟，首个无动力设施亲子游乐园，可以玩一整天，上次去过之后，小朋友一直惦记着，这回趁着放假有时间，再带他们去一下~上次停车在停车场，没有注意到这一个墙绘，感觉拍照也很好看！放假一说出去玩小朋友就很兴奋，来早了，还没开园随手拍的小孩子超爱的游玩点冒险乐园：儿童户外攀爬设施包场的感觉这一块够他们玩1个小时吧上次我也爬了一下，真累，大人还是适合在下面坐着等就行带小朋友出门玩，顺便找点童年回忆，跷跷板萝卜城堡：大滑梯，跷跷板这个像不像在头上比心，哈哈哈哈哈早上刚下过雨，有点湿悠然湖：家庭亲子游必玩，小孩子是开心了，大人就当锻炼身体了等下工作人员来开船大鱼船：海盗船造型，假装当船长，旋转滑梯在更里面一些的地方，避开一来是入园的人流，你就能拍到这种空空的照片竞速漂流：小竹筏漂流，锻炼胆量的好机会植物迷宫：儿童版整个乐园场地不大不小，小朋友几乎可以自由飞翔，不用担心找不到家长...乐园还设置了不少休息位置，让

0
实体标注结果为：
 OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO
本次标注的内容为：
 夏天已结束？疫情让一切改变，原本以为所有都在控制范围内，没想到，由五月广州开始爆发后，全国陆续又出现变种

温德姆酒店
浪漫海岸国际旅游度假区
0
实体标注结果为：
 BIIIIIIIIIIOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOBIIIIOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOBIIIIIIIIIIBIIIIIIIIIIBIIIIIIIIIIOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOBIIIIOOOOOOOBIIIIIIIIIIBIIIIIIIIIIBIIIIIIIIIIOOOOOOOOOOOOOOOOOOOOOOOOBIIIIIIIIIIOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOBIIIIIIIIIIOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOBIIIIIIIIIIBIIIIIIIIIIBIIIIIIIIIIBIIIIIIIIIIBIIIIIIIIIIOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOBIOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOBIIIIIIIIIIBIIIIOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOBIIIIIIIIIIBIIIIIIIIII

In [30]:
df_yj['新实体编码'] = text_code_yj
df_yj.head()

,游记ID,城市,游记标题,发布时间,出行天数,人物,人均费用,正文,人均费用_,实体编码,标注好的正文,新实体编码
0,游记攻略-001,茂名,2021-1-1 一路走来﻿皆有味道 ﻿送鼠迎牛庆贺元旦,2021-01-02 10:49,1 天,和朋友,180RMB,一元复始，万象更新，庆祝元旦猪杂先生：庆祝元旦，从早餐美味开始油甘窝，黑马国际俱乐部金塘，谭...,180.0,OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...,一元复始，万象更新，庆祝元旦猪杂先生：庆祝元旦，从早餐美味开始油甘窝，黑马国际俱乐部金塘，谭...,OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...
1,游记攻略-002,茂名,水东忠良街游记,2021-01-04 07:44,1 天,一个人,20RMB,水东城，位于南海水东湾畔，面积49.2平方公里，是广东省茂名市电白区区府所在地。水东原为滨海...,20.0,OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...,水东城，位于南海水东湾畔，面积49.2平方公里，是广东省茂名市电白区区府所在地。水东原为滨海...,OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...
2,游记攻略-003,茂名,水东滨海游记,2021-01-04 21:00,1 天,和朋友,20RMB,前言水东，是广东省茂名市电白区区府所在地。水东原为滨海渔村，明代开始逐渐发展成为电白的最大商...,20.0,OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...,前言水东，是广东省茂名市电白区区府所在地。水东原为滨海渔村，明代开始逐渐发展成为电白的最大商...,OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...
3,游记攻略-004,茂名,2021-1-2 借问乡旅何处好 高佬推荐杏花村,2021-01-02 21:24,1 天,和朋友,100RMB,元旦日后，阳光静好，迎君来储良杏花村佳话，琴瑟永和谐分界储良，杏花村，莫五姑的家乡。莫秀英（...,100.0,OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...,元旦日后，阳光静好，迎君来储良杏花村佳话，琴瑟永和谐分界储良，杏花村，莫五姑的家乡。莫秀英（...,OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...
4,游记攻略-005,茂名,2021-1-10 湖艳藏龙湾深埇大地大王岭绿野仙踪 南盛过山合太阳部落上宾塘知了书院,2021-01-11 08:58,1 天,和朋友,280RMB,今日行程：茂名【幸福大院出发】——油甘窝——化州平垌【湖艳村，藏龙湾】——南盛山合【太阳部落...,280.0,OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...,今日行程：茂名【幸福大院出发】——油甘窝——化州平垌【湖艳村，藏龙湾】——南盛山合【太阳部落...,OOOOOOOOOOOOOOOOOBIIOOOOOOOOOOOBIIOOOOOOOOOOOO...


In [31]:
df_yj.to_csv('data/新游记攻略.csv', index=False)

## 提取所有实体

### 读取文档

In [11]:
df_1 = pd.read_csv('data/新游记攻略.csv')
df_2 = pd.read_csv('data/酒店评论.csv')
df_3 = pd.read_csv('data/景区评论.csv')
df_4 = pd.read_csv('data/餐饮评论.csv')

In [12]:
df_1.head()

,游记ID,城市,游记标题,发布时间,出行天数,人物,人均费用,正文,人均费用_,实体编码,标注好的正文,新实体编码
0,游记攻略-001,茂名,2021-1-1 一路走来﻿皆有味道 ﻿送鼠迎牛庆贺元旦,2021-01-02 10:49,1 天,和朋友,180RMB,一元复始，万象更新，庆祝元旦猪杂先生：庆祝元旦，从早餐美味开始油甘窝，黑马国际俱乐部金塘，谭...,180.0,OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...,一元复始，万象更新，庆祝元旦猪杂先生：庆祝元旦，从早餐美味开始油甘窝，黑马国际俱乐部金塘，谭...,OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...
1,游记攻略-002,茂名,水东忠良街游记,2021-01-04 07:44,1 天,一个人,20RMB,水东城，位于南海水东湾畔，面积49.2平方公里，是广东省茂名市电白区区府所在地。水东原为滨海...,20.0,OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...,水东城，位于南海水东湾畔，面积49.2平方公里，是广东省茂名市电白区区府所在地。水东原为滨海...,OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...
2,游记攻略-003,茂名,水东滨海游记,2021-01-04 21:00,1 天,和朋友,20RMB,前言水东，是广东省茂名市电白区区府所在地。水东原为滨海渔村，明代开始逐渐发展成为电白的最大商...,20.0,OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...,前言水东，是广东省茂名市电白区区府所在地。水东原为滨海渔村，明代开始逐渐发展成为电白的最大商...,OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...
3,游记攻略-004,茂名,2021-1-2 借问乡旅何处好 高佬推荐杏花村,2021-01-02 21:24,1 天,和朋友,100RMB,元旦日后，阳光静好，迎君来储良杏花村佳话，琴瑟永和谐分界储良，杏花村，莫五姑的家乡。莫秀英（...,100.0,OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...,元旦日后，阳光静好，迎君来储良杏花村佳话，琴瑟永和谐分界储良，杏花村，莫五姑的家乡。莫秀英（...,OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...
4,游记攻略-005,茂名,2021-1-10 湖艳藏龙湾深埇大地大王岭绿野仙踪 南盛过山合太阳部落上宾塘知了书院,2021-01-11 08:58,1 天,和朋友,280RMB,今日行程：茂名【幸福大院出发】——油甘窝——化州平垌【湖艳村，藏龙湾】——南盛山合【太阳部落...,280.0,OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...,今日行程：茂名【幸福大院出发】——油甘窝——化州平垌【湖艳村，藏龙湾】——南盛山合【太阳部落...,OOOOOOOOOOOOOOOOOBIIOOOOOOOOOOOBIIOOOOOOOOOOOO...


### 实体识别函数

In [48]:
# 实体识别
def find_entity(conts=None, codes=None):
    entitys = set()
    for cont, code in zip(conts, codes):
        entity = ''
        w = list(cont)
        c = list(code)
        for i in range(len(w)):
            if c[i] == 'B' or c[i] == 'I':
                entity += w[i]
                # 判断I后面是否新的实体开始，是则添加前一个实体
                try:
                    if c[i+1] == 'B':
                        entitys.add(entity)
                        entity = ''
                except:
                    pass
            else:
                if entity != '':
                    entitys.add(entity)
                entity = ''
    return list(entitys)

In [49]:
entity1 = find_entity(df_1['正文'].astype(str), df_1['新实体编码'].astype(str))
entity2 = df_2['酒店名称'].tolist()
entity3 = find_entity(df_3['评论内容'].astype(str), df_3['实体编码'].astype(str))
entity4 = find_entity(df_4['评论内容'].astype(str), df_4['实体编码'].astype(str))

In [52]:
entity3

['广州海上丝绸之路博物馆',
 '红树林',
 '海滨公园',
 '雷祖祠',
 '麻斜岛',
 '海滩',
 '十贤祠',
 '阳江海上丝绸之路博物馆',
 '水东湾',
 '瞭望厅',
 '国家山地公园',
 '赤坎老街',
 '三元塔',
 '玻璃桥',
 '望海楼',
 '西湖公园',
 '摩天轮',
 '雷州西湖',
 '渔岛度假村',
 '湛江大桥',
 '雷州古城',
 '湖光岩',
 '露天矿生态公园',
 '海陵岛',
 '博物馆',
 '杏岛',
 '龙湾生态旅游区',
 '十里军港',
 '东南码头',
 '游乐场']

In [54]:
all_entity = list(set(entity1 + entity2 + entity3 + entity4))

### 保存所有实体

In [56]:
df_entity = pd.DataFrame({'实体': all_entity})
df_entity.to_csv('data/所有实体.txt')
df_entity.to_csv('data/所有实体.csv', index=False)

In [61]:
def log_sum(n=None, sl=None):
    s = 0
    for i in range(n):
        s += (sl[i] * (-np.log2(sl[i])))
    return s

In [73]:
s1 = log_sum(2, [3/5, 2/5])

In [74]:
s1

0.9709505944546686